In [1]:
import requests
import pandas as pd

#### [Gate IO API](https://www.gate.io/docs/developers/apiv4/en/#spot)

In [2]:
token = 'DDIM'

In [3]:
host = "https://api.gateio.ws"
prefix = "/api/v4"
headers = {'Accept': 'application/json', 'Content-Type': 'application/json'}

#### Get [historical market trades](https://www.gate.io/docs/developers/apiv4/en/#retrieve-market-trades)

In [4]:
url = '/spot/trades'
query_param = f'currency_pair={token}_USDT&limit=1000'

r = requests.request('GET', host + prefix + url + "?" + query_param, headers=headers)
trades_df = pd.DataFrame(r.json())

In [5]:
trades_convert = {
    'id': int,
    'create_time': int,
    'amount': float,
    'price': float,
    'sequence_id': int,
}

trades_df = trades_df.astype(trades_convert)

In [6]:
trades_df['dt'] = trades_df['create_time'].apply(lambda x: pd.to_datetime(x, unit='s'))

In [7]:
trades_df.set_index('dt', inplace=True)

In [8]:
trades_df.sort_index(inplace=True)

In [9]:
trades_df['notional'] = trades_df['amount'] * trades_df['price']

In [10]:
trades_df.tail()

,id,create_time,create_time_ms,currency_pair,side,amount,price,sequence_id,notional
dt,,,,,,,,,
2023-11-25 09:02:58,6731436479,1700902978,1700902978164.375000,DDIM_USDT,sell,8.903000,2.010,151787,17.895030
2023-11-25 09:03:21,6731439146,1700903001,1700903001161.722000,DDIM_USDT,sell,3.683219,2.021,151788,7.443786
2023-11-25 09:03:38,6731441160,1700903018,1700903018109.236000,DDIM_USDT,buy,1.619000,2.020,151789,3.270380
2023-11-25 09:05:11,6731450051,1700903111,1700903111147.700000,DDIM_USDT,buy,2.380115,2.023,151790,4.814974
2023-11-25 09:05:42,6731453439,1700903142,1700903142120.583000,DDIM_USDT,sell,6.745000,2.020,151791,13.624900


#### Get [orderbook snapshot](https://www.gate.io/docs/developers/apiv4/en/#retrieve-order-book)

In [11]:
url = '/spot/order_book'
query_param = f'currency_pair={token}_USDT'

r = requests.request('GET', host + prefix + url + "?" + query_param, headers=headers)
df = pd.DataFrame(r.json())

In [12]:
df.head()

,current,update,asks,bids
0,1700903157102,1700903148678,"[2.0407, 3.431]","[1.9976, 3.503]"
1,1700903157102,1700903148678,"[2.047, 2.488]","[1.985, 47.743]"
2,1700903157102,1700903148678,"[2.048, 43.264]","[1.984, 36.524]"
3,1700903157102,1700903148678,"[2.178, 7.445]","[1.982, 3.2201]"
4,1700903157102,1700903148678,"[2.179, 2.512]","[1.98, 13.97]"


### How to detect Fake Volume

#### Method 1: Connect to their Websocket (Both [Orderbook](https://www.gate.io/docs/developers/apiv4/ws/en/#order-book-channel) & [Public Trades](https://www.gate.io/docs/developers/apiv4/ws/en/#public-trades-channel) Channel)

- Will skip here since it's complicated to combine 2 threads & compare together here.

#### Method 2: Frequently Call Orderbook API (let's say every 5 seconds) & record the snapshots, compare with the trades history.

- If you see both bids & asks didn't change (can focus on best bid & best ask first), but trades happened. It is usually a fake volume printed.

In [13]:
def fetch_orderbook_snapshot() -> pd.DataFrame:
    url = '/spot/order_book'
    query_param = f'currency_pair={token}_USDT'

    r = requests.request('GET', host + prefix + url + "?" + query_param, headers=headers)
    df = pd.DataFrame(r.json())

    return df

In [ ]:
# Record orderbook snapshot every 5 seconds.
import time

for _ in range(120):
    ob_snapshot_df = fetch_orderbook_snapshot()
    last_update_ts = ob_snapshot_df['update'].unique()[0]
    ob_snapshot_df.to_csv(f'./{token}_ob_snapshot_{last_update_ts}.csv')
    time.sleep(5)


In [14]:
# After recording, you can compare the best bid & best ask & see if from the trades API, they have transctions, left by readers.